In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
import xgboost as xgb
warnings.filterwarnings("ignore")
%matplotlib inline

#导入经过预处理后的训练数据
train=pd.read_csv('G:/数据/train_clean.csv')

from sklearn.model_selection import KFold

#分离数据集，并剔除无关变量
x_train=train.drop(['isDefault','id','issueDate'],axis=1)
x_train=x_train.loc[:, :'n14']
y_train=train['isDefault']

#使用astype函数对grade,subgrade,employmentlength进行数据类型转换
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
a=['grade','subGrade','employmentLength','earliesCreditLine']
for name in a:
    for col1 in list(x_train):
        if col1 == name:
            x_train[col1]=lbl.fit_transform(x_train[col1].astype(str))

print(x_train.shape,y_train.shape)

#进行五折交叉检验
folds=5
seed=2020
kf=KFold(n_splits=folds,shuffle=True,random_state=seed)

from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

#对训练集数据进行划分，分成训练集和验证集
x_train_split,x_val,y_train_split,y_val=train_test_split(x_train,y_train,test_size=0.2)
train_matrix=xgb.DMatrix(x_train_split,label=y_train_split)
valid_matrix=xgb.DMatrix(x_val,label=y_val)

#设置xgb参数
params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'gamma': 0,
        'min_child_weight': 1,
        'max_depth': 6,
        'lambda': 1,
        'subsample': 1,
        'colsample_bytree': 1,
        'colsample_bylevel': 1,
        'eta': 0.3,
        'tree_method': 'exact',
        'seed': 2020,
        'n_jobs': -1,
        "silent": True,
    }
watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
model=xgb.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
roc_auc = metrics.auc(fpr, tpr)
print('调参前xgboost单模型在验证集上的AUC：{}'.format(roc_auc))

(800000, 44) (800000,)
[23:17:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.70308	eval-auc:0.70269
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.78036	eval-auc:0.72631
Stopping. Best iteration:
[136]	train-auc:0.76739	eval-auc:0.72734

调参前xgboost单模型在验证集上的AUC：0.7273440629006396


In [4]:
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
xgb2 = XGBClassifier(max_depth=3,
                     learning_rate=0.1,
                     n_estimatores=1494,
                     objective='binary:logistic',
                     booster='gbtree',
                     n_jobs=4,
                     gamma=0,
                     min_child_weight=1,
                     subsample=0.8,
                     colsample_bytree=0.8,

grid_search.fit(x_train_split,y_train_split)

print('best_params:',grid_search.best_params_)
print('best_score:',grid_search.best_score_)

[23:23:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimatores } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:23:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimatores } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:24:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimatores } might not be used.

  This may not be accurate due to some parameters are only used in